In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime
from tqdm import tqdm

current_time = datetime.now().strftime('%y-%m-%d')

# 키워드를 입력받습니다.
txt = input("키워드 : ")

# 크롬 웹드라이버 실행
driver = webdriver.Chrome()

# 네이버로 이동
driver.get('http://www.naver.com')
time.sleep(3)

# 검색창에 키워드 입력 후 검색
element = driver.find_element(By.ID, "query")
element.send_keys(txt)
element.submit()

# 블로그 탭 클릭
blog_link = driver.find_element(By.PARTIAL_LINK_TEXT, "블로그")
blog_link.click()
time.sleep(3)

# 스크롤 함수 정의
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 스크롤 내리기 (필요에 따라 n 값을 조정하세요)
n = 0
for _ in range(n):
    scroll_down(driver)

# URL과 제목 리스트 생성
url_list = []
title_list = []

# 블로그 글 제목과 URL 수집
article_raw = driver.find_elements(By.CSS_SELECTOR, '.title_link')

for article in article_raw:
    url = article.get_attribute('href')
    title = article.text
    url_list.append(url)
    title_list.append(title)

# 수집한 URL과 제목을 DataFrame에 저장
df = pd.DataFrame({'주소': url_list, '제목': title_list})

# 크롤링한 내용을 저장할 '내용' 컬럼 추가
df['작성일자'] = None
df['태그'] = None
df['내용'] = None

# 스크랩할 페이지 수 (df의 길이만큼 반복합니다.)
number = len(df)

for i in tqdm(range(number)):
    
    url = df['주소'][i]  # DataFrame에서 URL을 가져옵니다.
    driver = webdriver.Chrome()
    driver.get(url)    
    time.sleep(3)

    try:
        # 블로그 본문 iframe으로 전환
        driver.switch_to.frame('mainFrame')

        # 제목 추출
        tit = driver.find_element(By.CSS_SELECTOR, '.se-module.se-module-text.se-title-text')
        title = tit.text

        # 날짜 및 시간 추출
        date = driver.find_element(By.CSS_SELECTOR, '.se_publishDate.pcol2')
        datetime = date.text

        df.at[i, '작성일자'] = datetime

        # 태그 추출
        tags = driver.find_elements(By.CSS_SELECTOR, '.wrap_tag .ell')

        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)

        tag_str = ' '.join(tag_list)

        # 크롤링한 태그 목록을 DataFrame의 '태그' 컬럼에 저장
        df.at[i, '태그'] = tag_str

        # 내용 추출
        contents = driver.find_elements(By.CSS_SELECTOR, '.se-module.se-module-text')
        
        content_list = []
        for content in contents:
            content_list.append(content.text)
        
        content_str = ' '.join(content_list)

        # 크롤링한 내용을 DataFrame의 '내용' 컬럼에 저장
        df.at[i, '내용'] = content_str

        print(f"{i}번째 글 크롤링 완료: {title}")
        
        driver.quit()

    except Exception as e:
        print(f"{i}번째 글에서 오류 발생: {e}")
        driver.quit()
        time.sleep(2)
        continue

# 크롤링한 내용을 포함한 DataFrame을 Excel 파일로 저장
df.to_excel(f"블로그내용크롤링 {current_time}.xlsx", index=False)

print("수집한 글 갯수:", len(df.dropna(subset=['내용'])))

driver.quit()

키워드 :  뮤지컬 시카고


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

0번째 글 크롤링 완료: 뮤지컬 시카고 후기 정선아 민경아 박건형 MD 디큐브링크아트센터 시야 기본 정보


  3%|██▊                                                                                | 1/30 [00:07<03:51,  7.98s/it]